# BERDL Data API Utilities

This notebook demonstrates how to use KBBERDLUtils to query the BERDL (Biological and Environmental Research Data Lake) API.

## Overview

BERDL provides SQL access to KBase data stored in Delta Lake format. KBBERDLUtils offers:
- **Database Introspection**: List databases, tables, and column schemas
- **SQL Query Execution**: Run arbitrary SQL queries against delta tables
- **Automatic Pagination**: Handle large result sets with built-in pagination
- **Connection Testing**: Verify API connectivity and authentication

## Prerequisites

- **KBase Token**: You need a valid KBase token with BERDL user role
- Token should be stored in `~/.kbase/token` or set programmatically

## Performance Notes

| Scenario | Cluster | Performance |
|----------|---------|-------------|
| Logged into BERDL JupyterHub | Personal | Faster |
| Not logged in / session expired | Shared | Slower |
| KBase app (service account) | Shared | Slower |

## 1. Setup

In [1]:
%run util.py

#print(kbberdlutil.get_token("berdl"))

/Users/chenry/Dropbox/Projects/KBUtilLib/src


[KBUtilLib] Failed to import rcsb_pdb_utils: ModuleNotFoundError: No module named 'aiohttp'


modelseedpy 0.4.2


2026-02-09 10:29:59,800 - __main__.BVBRCUtil - INFO - Loaded configuration from: /Users/chenry/.kbutillib/config.yaml
2026-02-09 10:29:59,801 - __main__.BVBRCUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2026-02-09 10:29:59,801 - __main__.BVBRCUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token


cobrakbase 0.4.0


2026-02-09 10:30:00,171 - __main__.BVBRCUtil - INFO - Notebook name: BERDLDataAPI
2026-02-09 10:30:00,172 - __main__.BVBRCUtil - INFO - Notebook environment detected
2026-02-09 10:30:00,174 - __main__.KBBERDLUtil - INFO - Loaded configuration from: /Users/chenry/.kbutillib/config.yaml
2026-02-09 10:30:00,175 - __main__.KBBERDLUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2026-02-09 10:30:00,175 - __main__.KBBERDLUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2026-02-09 10:30:00,176 - __main__.KBBERDLUtil - INFO - KBBERDLUtils initialized (API: https://hub.berdl.kbase.us/apis/mcp/delta)
2026-02-09 10:30:00,176 - __main__.KBBERDLUtil - INFO - Notebook name: BERDLDataAPI
2026-02-09 10:30:00,177 - __main__.KBBERDLUtil - INFO - Notebook environment detected
2026-02-09 10:30:00,179 - __main__.AICurationUtil - INFO - Loaded configuration from: /Users/chenry/.kbutillib/config.yaml
2026-02-09 10:30:00,179 - __main__.AICurationUtil - INFO - Loaded 0 tokens from /Use

## 2. Test Connection

Verify that we can connect to the BERDL API with our token:

In [ ]:
%run util.py

# Test the connection
result = kbberdlutil.test_connection()

if result["success"]:
    print("Connected successfully!")
    print(f"API URL: {result['api_url']}")
else:
    print(f"Connection failed: {result['message']}")
    print("\nMake sure you have:")
    print("  1. A valid KBase token in ~/.kbase/token")
    print("  2. BERDL user role assigned to your account")

## 3. Explore Available Databases

List all databases available in BERDL:

In [ ]:
%run util.py

# List all available databases
result = kbberdlutil.get_database_list()

if result["success"]:
    print(f"Found {result['count']} databases:\n")
    for db in result["databases"]:
        print(f"  - {db}")
else:
    print(f"Failed: {result['error']}")

## 4. Explore Tables in a Database

List all tables in a specific database:

In [ ]:
%run util.py

# List tables in kbase_genomes database
result = kbberdlutil.get_database_tables("kbase_genomes")

if result["success"]:
    print(f"Tables in '{result['database']}' ({result['count']} total):\n")
    for table in result["tables"]:
        print(f"  - {table}")
else:
    print(f"Failed: {result['error']}")

## 5. Explore Table Columns

Get column information for a specific table:

In [ ]:
%run util.py

# Get columns for the contig table
result = kbberdlutil.get_table_columns("kbase_genomes", "contig")

if result["success"]:
    print(f"Columns in {result['database']}.{result['table']}:\n")
    for col in result["columns"]:
        print(f"  - {col['name']}: {col['type']}")
else:
    print(f"Failed: {result['error']}")

## 6. Get Complete Database Schema

Get the full structure of databases and tables:

In [ ]:
%run util.py

# Get complete database structure
result = kbberdlutil.get_database_schema()

if result["success"]:
    print("Database Structure:\n")
    for db_name, tables in result["schema"].items():
        print(f"  {db_name}:")
        if isinstance(tables, list):
            for table in tables[:5]:  # Show first 5 tables
                print(f"    - {table}")
            if len(tables) > 5:
                print(f"    ... and {len(tables) - 5} more tables")
else:
    print(f"Failed: {result['error']}")

## 7. Get Schema with Column Details

Get schema with column details for a specific database:

In [ ]:
%run util.py

# Get schema with column details for a specific database
result = kbberdlutil.get_database_schema(database="kbase_genomes", include_columns=True)

if result["success"]:
    print("Schema with columns:\n")
    schema = result["schema"]
    # Display first table with its columns
    if schema:
        for table_name, columns in list(schema.items())[:2]:
            print(f"  {table_name}:")
            if isinstance(columns, list):
                for col in columns[:5]:
                    print(f"    - {col}")
else:
    print(f"Failed: {result['error']}")

## 8. SQL Queries - Contig Data

Execute custom SQL queries against BERDL:

In [ ]:
%run util.py

# Query contig data
sql = """
SELECT contig_id, gc_content, length
FROM kbase_genomes.contig
ORDER BY contig_id
"""

result = kbberdlutil.query(sql, limit=5)

if result["success"]:
    print(f"Retrieved {result['row_count']} contigs")
    print(f"Columns: {result['columns']}\n")
    for row in result["data"]:
        print(f"  {row['contig_id'][:30]}... | GC: {row.get('gc_content', 'N/A')} | Length: {row.get('length', 'N/A')}")
else:
    print(f"Query failed: {result['error']}")

## 9. SQL Queries - Reaction Names

Query ontology statements for SEED reaction names:

In [ ]:
%run util.py

# Query SEED reaction names from ontology
sql = """
SELECT
    subject as reaction_id,
    value as reaction_name
FROM kbase_ontology_source.statements
WHERE subject LIKE 'seed.reaction:%'
AND predicate = 'rdfs:label'
ORDER BY subject
"""

result = kbberdlutil.query(sql, limit=10)

if result["success"]:
    print(f"Found {result['row_count']} reaction labels:\n")
    for row in result["data"]:
        rxn_id = row["reaction_id"].replace("seed.reaction:", "")
        print(f"  {rxn_id}: {row['reaction_name']}")
else:
    print(f"Query failed: {result['error']}")

## 10. SQL Queries - Compound Names

Query ontology statements for SEED compound names:

In [ ]:
%run util.py

# Query SEED compound names
sql = """
SELECT
    subject as compound_id,
    value as compound_name
FROM kbase_ontology_source.statements
WHERE subject LIKE 'seed.compound:%'
AND predicate = 'rdfs:label'
ORDER BY subject
"""

result = kbberdlutil.query(sql, limit=10)

if result["success"]:
    print(f"Found {result['row_count']} compound labels:\n")
    for row in result["data"]:
        cpd_id = row["compound_id"].replace("seed.compound:", "")
        print(f"  {cpd_id}: {row['compound_name']}")
else:
    print(f"Query failed: {result['error']}")

## 11. Paginated Queries

For large result sets, use pagination to fetch all data:

In [ ]:
%run util.py

# Paginated query example - fetch larger result sets
result = kbberdlutil.paginate_query(
    sql="SELECT * FROM kbase_genomes.contig WHERE length > 5000 ORDER BY contig_id",
    page_size=100,
    max_pages=5  # Limit pages for demo
)

if result["success"]:
    print(f"Total rows retrieved: {result['row_count']}")
    print(f"Pages fetched: {result['pages_fetched']}")
    print(f"\nFirst 3 results:")
    for row in result["data"][:3]:
        print(f"  {row}")
else:
    print(f"Query failed: {result['error']}")

## 12. View Documentation

Print the full BERDL documentation:

In [ ]:
%run util.py

# View the full module documentation
kbberdlutil.print_docs()

## Summary

KBBERDLUtils provides a clean, focused API for accessing the BERDL data lake:

### Database Introspection
- `get_database_list()` - List all available databases
- `get_database_tables(database)` - List tables in a database
- `get_table_columns(database, table)` - Get column schema for a table
- `get_database_schema()` - Get complete database structure

### Query Execution
- `query(sql, limit, offset)` - Execute SQL queries
- `paginate_query(sql, page_size, max_pages)` - Fetch large result sets

### Utilities
- `test_connection()` - Verify API connectivity
- `print_docs()` - Display module documentation

### Performance Notes
- **JupyterHub Login**: Uses personal cluster (faster)
- **No Login / KBase Apps**: Uses shared cluster (slower)

### Resources
- [BERDL Swagger Docs](https://hub.berdl.kbase.us/apis/mcp/docs) - Full API reference
- [KBBERDLUtils Docs](../docs/modules/kb_berdl_utils.md) - Module documentation